In [1]:
import sys
sys.path.append('../')
from data import create_training_data, create_evaluation_data
from pipeline import run_model_training, run_hyperparameter_tuning, run_model_training_quick, run_predictions

In [2]:
df = create_training_data("2020-21")
eval_df = create_evaluation_data()

test data version 0.1 stable
test data version 0.1 stable


In [ ]:
best_hyperparams = run_hyperparameter_tuning(df)

In [ ]:
model = run_model_training(best_hyperparams, df)

In [5]:
preds = run_predictions(eval_df, "model_2021_06_09_16_35.pkl")

In [8]:
preds.head(n = 30)

,name,ict_index,bps,minutes,now_cost,predicted_points
311,Firmino,10.5,38,80,88,9.164640
139,Wood,21.8,35,254,69,8.838139
340,Mahrez,23.0,58,122,89,8.838139
334,Grealish,35.9,74,254,80,8.838139
323,Jota,25.7,47,187,76,8.838139
320,Alexander-Arnold,38.6,98,270,75,8.838139
316,Salah,42.8,81,270,125,8.838139
313,Mané,39.2,40,270,119,8.838139
175,Mount,21.4,54,261,75,8.838139
183,Lukaku,27.8,39,180,115,8.838139
